In [2]:
from transformers import ElectraModel, ElectraTokenizer

model = ElectraModel.from_pretrained("koelectra-base-v3-discriminator")
tokenizer = ElectraTokenizer.from_pretrained("koelectra-base-v3-discriminator")

Some weights of the model checkpoint at koelectra-base-v3-discriminator were not used when initializing ElectraModel: ['discriminator_predictions.dense.weight', 'discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.weight']
- This IS expected if you are initializing ElectraModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [6]:
import tensorflow as tf
from transformers import ElectraTokenizer, TFElectraForSequenceClassification
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd

# GPU 환경 설정
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # GPU 메모리 늘리기
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        # 특정 GPU 선택하기 (여러 개 중에서 원하는 GPU 번호를 선택)
        # tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
    except RuntimeError as e:
        print(e)

In [11]:
# 데이터프레임에서 데이터 읽어오기
dataset = pd.read_csv("./dataset/train_data.csv", encoding='UTF-8')  # 'dataset.csv'는 실제 데이터 파일의 이름에 맞게 변경해주세요

# 데이터 프레임에서 한국어 문장과 라벨을 가져옵니다.
sentences = dataset['sentence'].values
labels = dataset['sentiment'].values

In [52]:
# 문장과 감정 레이블로 분리
sentences = df['sentence'].values
labels = df['sentiment'].values

In [12]:
# 라벨을 One-Hot 인코딩으로 변환합니다.
label_map = {'분노': 0, '기쁨': 1, '불안': 2, '당황': 3, '슬픔': 4, '상처': 5}
labels = np.array([label_map[label] for label in labels])
labels = tf.keras.utils.to_categorical(labels, num_classes=6)

In [14]:
# koELECTRA 토크나이저 불러오기
tokenizer = ElectraTokenizer.from_pretrained("koelectra-base-v3-discriminator")

In [15]:
# 문장을 토큰화하고 시퀀스로 변환합니다.
sequences = [tokenizer.encode(sentence, padding='max_length', max_length=128, truncation=True) for sentence in sentences]

In [16]:
# 학습 데이터와 검증 데이터로 나눕니다.
train_data, val_data, train_labels, val_labels = train_test_split(
    sequences, labels, test_size=0.2, random_state=42
)

In [17]:
tf.config.experimental.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [ ]:
# CUDA 11.7 버전의 GPU를 사용하여 모델 불러오기
with tf.device('/GPU:0'):
    model = TFElectraForSequenceClassification.from_pretrained("koelectra-base-v3-discriminator", num_labels=6)

In [22]:
tf.__version__

'1.14.0'